In [47]:
import pandas as pd
pd.set_option('display.float_format', str)

In [7]:
# Import conditions file
cond_file = "https://github.com/spiros/nkua-health-data-science/raw/master/data/dest/conditions.csv.gz"
df_cond = pd.read_csv(cond_file)
print(df_cond.shape)

(38100, 5)


In [8]:
df_cond.head(3)

,START,STOP,PATIENT,ENCOUNTER,CODE
0,2013-06-24,2013-07-02,c1f1fcaa-82fd-d5b7-3544-c8f9708b06a8,0b2794bd-ec2b-d34f-0610-2523b3b7fcf0,10509002.0
1,2016-02-27,2016-03-14,c1f1fcaa-82fd-d5b7-3544-c8f9708b06a8,a6d818dd-0983-fd1c-eefa-3d2295532c45,283371005.0
2,2016-08-11,2016-08-22,c1f1fcaa-82fd-d5b7-3544-c8f9708b06a8,36d2e781-4655-0a11-1f70-c69856e02019,444814009.0


In [16]:
# Print the data types
df_cond.dtypes

START        datetime64[ns]
STOP                 object
PATIENT              object
ENCOUNTER            object
CODE                float64
dtype: object

In [17]:
# Convert the START and STOP columns to datetime objects.
# Check how many dates of admission were set to NA because
# they were incorrect.

df_cond['START'] = pd.to_datetime(
    df_cond['START'],
    errors='coerce'
)

df_cond['START'] = pd.to_datetime(
    df_cond['START'],
    errors='coerce'
)

print(df_cond[df_cond['START'].isna()])

Empty DataFrame
Columns: [START, STOP, PATIENT, ENCOUNTER, CODE]
Index: []


In [ ]:
# Set the dtype of the SNOMED code to a string


In [23]:
# Count the number of unique diagnosis codes.
df_cond.CODE.nunique()

204

In [24]:
# Count the number of unique patients codes.
df_cond.PATIENT.nunique()

1147

In [25]:
# Count the number of unique encounters
df_cond.ENCOUNTER.nunique()

26904

In [26]:
# Counte number of encounters that are missing a diagnosis code.
print(df_cond[df_cond['CODE'].isna()].ENCOUNTER.nunique())
print(df_cond[df_cond['CODE'].isna()].PATIENT.nunique())

20
20


In [32]:
# Import the SNOMED dictionary
snomed_file = "https://github.com/spiros/nkua-health-data-science/raw/master/data/dest/dictionary_snomed.csv"
df_snomed = pd.read_csv(snomed_file)
print(df_snomed.shape)

(204, 2)


In [33]:
df_snomed.head(3)

,CODE,DESCRIPTION
0,10509002,Acute bronchitis (disorder)
1,283371005,Laceration of forearm
2,444814009,Viral sinusitis (disorder)


In [34]:
df_snomed.CODE.nunique()

202

In [37]:
# Count the number of rows where the CODE 
# or DESCRIPTION values are missing
df_snomed[df_snomed.CODE.isna()].shape

(0, 2)

In [38]:
df_snomed[df_snomed.DESCRIPTION.isna()].shape

(0, 2)

In [59]:
# Merge SNOMED dictionary with the conditions table.
# Use a left merge on the conditions table as to retain
# all information on the encounters that patients have had.
df_cond_merged = df_cond.merge(
    df_snomed,
    how='left',
    left_on='CODE',
    right_on='CODE',
    indicator=True
)

In [63]:
# Display invalid SNOMED codes and the number of patients and hospitalizations
# that these codes affect.

print(df_cond_merged._merge.value_counts())
print(df_cond_merged[df_cond_merged['DESCRIPTION'].isna()].PATIENT.nunique())
print(df_cond_merged[df_cond_merged['DESCRIPTION'].isna()].ENCOUNTER.nunique())

_merge
both          38107
left_only        63
right_only        0
Name: count, dtype: int64
25
26


In [66]:
# Display the top ten most common diagnoses.
g = ['CODE', 'DESCRIPTION']
df_cond_merged.groupby(g).size().reset_index(name='count').sort_values('count', ascending=False).head(25)

,CODE,DESCRIPTION,count
89,160903007.0,Full-time employment (finding),13793
57,73595000.0,Stress (finding),5131
90,160904001.0,Part-time employment (finding),2422
166,422650009.0,Social isolation (finding),1240
180,444814009.0,Viral sinusitis (disorder),1235
167,423315002.0,Limited social contact (finding),1197
190,741062008.0,Not in labor force (finding),1076
187,706893006.0,Victim of intimate partner abuse (finding),818
98,195662009.0,Acute viral pharyngitis (disorder),678
4,10509002.0,Acute bronchitis (disorder),571


In [65]:
# Display the bottom ten most common diagnoses.
df_cond_merged.groupby(g).size().reset_index(name='count').sort_values('count', ascending=True).head(15)

,CODE,DESCRIPTION,count
0,1734006.0,Fracture of the vertebral column with spinal c...,1
61,76916001.0,Spina bifida occulta (disorder),1
70,86175003.0,Injury of heart (disorder),1
88,157265008.0,Dislocation of hip joint (disorder),1
92,161622006.0,History of lower limb amputation (situation),1
96,190905008.0,Cystic Fibrosis,1
109,225444004.0,At risk for suicide (finding),1
30,45816000.0,Pyelonephritis,1
116,234466008.0,Acquired coagulation disorder (disorder),1
24,40095003.0,Injury of kidney (disorder),1
